In [1]:
# Import modules
import pandas as pd
import numpy as np
import pickle
import json

In [2]:
# Import sklearn modules
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import FunctionTransformer,OneHotEncoder


In [3]:
from xgboost import XGBClassifier

In [4]:
# Define categories and category feature columns
categories = ['beauty', 'fashion', 'mobile']
category_feature_columns = {'beauty':[ 'Brand', 'Colour_group', 'Benefits', 'Product_texture', 'Skin_type'], 
                   'fashion': ['Collar Type', 'Sleeves', 'Pattern', 'Fashion Trend', 'Clothing Material'],
                   'mobile': ['Operating System', 'Features',
       'Network Connections', 'Memory RAM', 'Brand', 'Warranty Period',
       'Storage Capacity', 'Color Family', 'Phone Model', 'Camera',
       'Phone Screen Size']}

# Define all utility functions here

In [5]:
def train_test_data(df, label, test_size):
    '''Prepare training and test data'''
    df = df.dropna(subset = [label])
    df_train = df.drop(columns = [label, 'itemid', 'image_path'])
    df_train.fillna(9999, inplace = True)
    X = df_train
    y = df[label]
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=test_size)
    return X_train,X_test,y_train,y_test

In [28]:
def train_test_data_title(df, label, test_size):
    '''Prepare training and test data'''
    df = df[['title', label]]
    df = df.dropna()
    X = df['title']
    y = df[label]
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=test_size)
    return X_train,X_test,y_train,y_test

In [6]:
def get_title(df):
    return df['title']

def get_img_column(df):
    return df.loc[:, df.columns.str.contains('^img')]

def get_non_title(df):
    '''Select all other feature columns'''
    return df.loc[:, ~df.columns.str.contains('^img')].drop(columns = ['title'])

In [266]:
get_img_column(df_mobile_train_img)


,img_feature,img_1,img_2,img_3,img_4,img_5,img_6,img_7,img_8,img_9,...,img_55,img_56,img_57,img_58,img_59,img_60,img_61,img_62,img_63,img_64
0,"[-0.002794798077051632, 0.0016848809626056809,...",-0.002795,0.001685,0.000311,-0.001125,0.004162,-0.000753,0.001901,-0.000858,-6.360886e-05,...,-0.000193,0.000992,0.001299,-0.000006,-0.001077,-0.000003,-0.001733,0.001720,0.001850,-0.001584
1,"[-0.002762159331473762, 0.0015487885942769132,...",-0.002762,0.001549,0.000844,-0.000401,0.002943,-0.000488,0.001059,-0.001807,1.612092e-04,...,-0.001912,0.000128,0.000843,-0.001290,-0.000159,-0.000258,-0.001315,0.001206,0.001254,-0.001469
2,"[-0.0018374733389897994, 0.0003378050104713345...",-0.001837,0.000338,0.000497,0.002438,0.001325,0.001257,0.001925,-0.001533,-1.911804e-04,...,-0.000786,0.000419,0.001106,-0.001306,-0.001100,0.000832,0.000752,-0.003119,0.000986,0.000871
3,"[-0.0026619780970929875, 0.001857001632737231,...",-0.002662,0.001857,0.000061,-0.000265,0.005522,-0.000642,0.002293,-0.000368,-4.226854e-04,...,-0.000135,0.002298,0.000370,-0.000230,-0.001024,-0.000727,-0.000983,-0.000437,0.000039,-0.000649
4,"[-0.0006973030780710945, -0.001820069781310920...",-0.000697,-0.001820,0.001341,0.002729,-0.003835,-0.000485,-0.001177,-0.000061,5.456535e-03,...,0.000240,0.000294,0.001366,0.000087,-0.000818,0.001540,0.000661,-0.005062,-0.000468,0.002798
5,"[0.0036478649080731243, -0.001901879134261641,...",0.003648,-0.001902,0.003672,-0.002234,0.001273,0.005493,0.003193,0.002771,-5.081179e-07,...,-0.003056,-0.008503,-0.003730,0.003586,0.002681,0.003029,0.000711,-0.001465,-0.003868,-0.001607
6,"[-0.0004921146511924819, 0.0003431806973149874...",-0.000492,0.000343,-0.001423,0.003949,0.004435,0.001856,-0.000465,0.003112,-2.817635e-03,...,0.000165,-0.000417,-0.002100,-0.002441,0.002099,0.002507,0.002037,-0.002396,0.000608,-0.002925
7,"[0.00011239756663277496, 0.0012084894069419923...",0.000112,0.001208,0.003342,0.001708,0.000932,0.001936,-0.001747,0.004960,-5.854262e-03,...,-0.002509,0.001974,0.003889,0.000995,-0.001005,0.003817,-0.000590,0.000772,0.000640,-0.002362
8,"[-0.0025110689144115382, 0.0015723645642389972...",-0.002511,0.001572,0.000375,0.000214,0.003224,0.000303,0.001145,-0.002027,9.091343e-04,...,0.000399,-0.000099,-0.000534,-0.001502,-0.000370,-0.001803,0.000769,-0.000281,-0.000080,0.000705
9,"[0.00034649936120814424, 0.0010383227234739933...",0.000346,0.001038,0.002610,0.002588,-0.002028,0.000753,-0.005002,0.002830,-3.756590e-03,...,0.003809,0.001103,0.000290,0.001132,0.000334,0.002347,-0.001637,0.001573,0.000701,0.001485


In [264]:
get_non_title(df_mobile_train)

,Operating System,Features,Network Connections,Memory RAM,Brand,Warranty Period,Storage Capacity,Color Family,Phone Model,Camera,Phone Screen Size
0,NaN,NaN,NaN,NaN,2.0,NaN,NaN,12.0,1526.0,NaN,NaN
1,NaN,NaN,NaN,NaN,2.0,NaN,3.0,12.0,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,43.0,13.0,NaN,12.0,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,43.0,13.0,NaN,12.0,NaN,NaN,NaN
4,6.0,NaN,0.0,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0.0,NaN,NaN,33.0,NaN,NaN,NaN,376.0,NaN,NaN
6,NaN,NaN,NaN,NaN,43.0,13.0,NaN,10.0,NaN,NaN,NaN
7,NaN,NaN,0.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,0.0,NaN,NaN,51.0,2.0,NaN,12.0,NaN,NaN,NaN
9,NaN,NaN,1.0,NaN,43.0,NaN,NaN,NaN,559.0,NaN,NaN


# Train SGD model

In [273]:
def build_SGD_model_img():
    """
    Build the SGD model
    """ 

    # Build Transformer
    
    transformer = FeatureUnion(
                            transformer_list = [
                               ('title_processed', 
                                Pipeline([
                                    ('extract_title', FunctionTransformer(get_title, validate = False)),
                                    ('title_vect', CountVectorizer()),
                                    ('title_tfidf', TfidfTransformer())
                                    ])),
                               ('other_columns', 
                                Pipeline([
                                    ('extract_others',FunctionTransformer(get_non_title, validate = False)),
                                    ('one_hot', OneHotEncoder(handle_unknown='ignore'))])),
                                ('img_column',
                                Pipeline([
                                    ('extract_img', FunctionTransformer(get_img_column, validate = False))
                                ]))
                                ]
                              )
    
    # Build pipeline
    text_clf = Pipeline([('transformer', transformer),
                         ('clf', SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3, class_weight='balanced'))])


    # Define parameters for grid search
    parameters = {'transformer__title_processed__title_vect__ngram_range': [(1, 1), (1, 2)],
                'transformer__title_processed__title_tfidf__use_idf': (True, False),
                'clf__alpha': (1e-1, 1e-5)}


    # Grid search across our parameters, scoring by accuracy
    gs_clf = GridSearchCV(estimator=text_clf, param_grid=parameters, cv=5, iid=False, n_jobs=-1, scoring='accuracy')

    return gs_clf

In [274]:
def build_SGD_model():
    """
    Build the SGD model
    """ 

    # Build Transformer
    
    transformer = FeatureUnion(
                            transformer_list = [
                               ('title_processed', 
                                Pipeline([
                                    ('extract_title', FunctionTransformer(get_title, validate = False)),
                                    ('title_vect', CountVectorizer()),
                                    ('title_tfidf', TfidfTransformer())
                                    ])),
                               ('other_columns', 
                                Pipeline([
                                    ('extract_others',FunctionTransformer(get_non_title, validate = False)),
                                    ('one_hot', OneHotEncoder(handle_unknown='ignore'))]))
                                ]
                              )
    
    # Build pipeline
    text_clf = Pipeline([('transformer', transformer),
                         ('clf', SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3, class_weight='balanced'))])


    # Define parameters for grid search
    parameters = {'transformer__title_processed__title_vect__ngram_range': [(1, 1), (1, 2)],
                'transformer__title_processed__title_tfidf__use_idf': (True, False),
                'clf__alpha': (1e-1, 1e-5)}


    # Grid search across our parameters, scoring by accuracy
    gs_clf = GridSearchCV(estimator=text_clf, param_grid=parameters, cv=5, iid=False, n_jobs=-1, scoring='accuracy')

    return gs_clf

In [275]:

### Train the SGD model
def train_test_SGD_model(category, target_column, X_train, y_train):
    SGD_model = build_SGD_model()
    SGD_model.fit(X_train, y_train)
    # Evaluation
    best_params = SGD_model.best_params_
    predicted = SGD_model.predict(X_test)
    print('Best Parameters: ', best_params)
    print('Train Set Accuracy: ', round(SGD_model.best_score_, 2))
    print('Test Set Accuracy: ', round(np.mean(predicted == y_test), 2))
    
    with open ('SGD_clf_18032019_{}_{}.pkl'.format(category, target_column), 'wb') as f:
        pickle.dump(SGD_model, f)
        
    return SGD_model
#     print('Train set Accuracy (cross val): ', cross_val_score(SGD_model, X_train, y_train, cv=4, scoring='accuracy'))
#     print(classification_report(y_test, predicted))
#     print(confusion_matrix(y_test, predicted))
#     with open ('SGD_clf_28022019_{}.pkl'.format(label), 'wb') as f:
#         pickle.dump(SGD_model, f)

## Train SGD for beauty 

In [276]:
beauty_feature_columns = [ 'Brand', 'Colour_group', 'Benefits', 'Product_texture', 'Skin_type']

df_beauty_train = pd.read_csv('../data/beauty_data_info_train_competition.csv')


for column in beauty_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_beauty_train, column, 0.2)
    SGD_model = train_test_SGD_model('beauty', column, X_train, y_train)

Now processing for column: Brand


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.99
Test Set Accuracy:  0.98
Now processing for column: Colour_group
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.87
Test Set Accuracy:  0.87
Now processing for column: Benefits
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.86
Test Set Accuracy:  0.87
Now processing for column: Product_texture
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.99
Test Set Accuracy:  0.99
Now processing

## Train SGD for mobile

In [277]:


for column in mobile_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_mobile_train_img, column, 0.2)
    SGD_model = train_test_SGD_model('mobile', column, X_train, y_train)

Now processing for column: Operating System


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 2 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.97
Test Set Accuracy:  0.97
Now processing for column: Features
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.79
Test Set Accuracy:  0.82
Now processing for column: Network Connections
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.91
Test Set Accuracy:  0.92
Now processing for column: Memory RAM
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.9
Test Set Accuracy:  0.91
Now processing 

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.99
Test Set Accuracy:  0.98
Now processing for column: Warranty Period
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.79
Test Set Accuracy:  0.78
Now processing for column: Storage Capacity
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.95
Test Set Accuracy:  0.95
Now processing for column: Color Family


...

Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.82
Test Set Accuracy:  0.83
Now processing for column: Phone Model


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.93
Test Set Accuracy:  0.94
Now processing for column: Camera


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': False, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.7
Test Set Accuracy:  0.69
Now processing for column: Phone Screen Size
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.75
Test Set Accuracy:  0.76


In [121]:
### Adding img 

mobile_img = np.load('../additional_data/mobile_img_64.npy')

In [ ]:
column_name = []
for i in range(1,65):
    column_name.append('img_{}'.format(i))
    
df_img = pd.DataFrame(mobile_img, columns = column_name)

In [268]:
df_mobile_train_img = pd.concat([df_mobile_train, df_img], axis=1, sort=False)

## Train SGD for fashion

In [278]:
fashion_feature_columns = ['Collar Type', 'Sleeves', 'Pattern', 'Fashion Trend', 'Clothing Material']


df_fashion_train = pd.read_csv('../data/fashion_data_info_train_competition.csv')

for column in fashion_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_fashion_train, column, 0.2)
    SGD_model = train_test_SGD_model('fashion', column, X_train, y_train)

Now processing for column: Collar Type
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.83
Test Set Accuracy:  0.83
Now processing for column: Sleeves
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.98
Test Set Accuracy:  0.98
Now processing for column: Pattern
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.93
Test Set Accuracy:  0.94
Now processing for column: Fashion Trend
Best Parameters:  {'clf__alpha': 1e-05, 'transformer__title_processed__title_tfidf__use_idf': True, 'transformer__title_processed__title_vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.91
Test Set Ac

In [281]:
df_mobile_train['title']

0         apple iphone 4s back glass spare part original...
1                                      iphone 4s 64gb white
2                           samsung sm b310e piton dual sim
3             samsung caramel gt e1272 dual sim 32 mb putih
4                        garskin sony experia z z1 z2 ultra
5                            lcd xiaomi redmi 4+touchscreen
6              samsung caramel gt e1272 dual sim 32mb black
7                                             iphone 4g 8gb
8         blackberry torch 1 9800 gsm garansi distributo...
9                               samsung keystone 3 sm b109e
10                      samsung galaxy j5 j 500g 8 gb hitam
11                 samsung galaxy j1 mini sm j105 8gb white
12                        iphone 5 white 16gb fullset mulus
13                              lenovo a 6000 se 1 16 white
14                                        samsung galaxy j1
15                             keypad blackberry 9360 hitam
16                                     x

# Using SGD model to predict testing dataset

In [175]:
df_fashion_val = pd.read_csv('../data/fashion_data_info_val_competition.csv')
X_test = df_fashion_val['title']

for column in fashion_feature_columns:
    with open('SGD_clf_02032019_fashion_{}.pkl'.format(column), 'rb') as f:
        SGD_clf = pickle.load(f)
    predicted = SGD_clf.predict(X_test)
    df_fashion_val[column] = predicted
    

In [92]:
df_mobile_val = pd.read_csv('../data/mobile_data_info_val_competition.csv')
X_test = df_mobile_val['title']

for column in mobile_feature_columns:
    with open('SGD_clf_02032019_mobile_{}.pkl'.format(column), 'rb') as f:
        SGD_clf = pickle.load(f)
    predicted = SGD_clf.predict(X_test)
    df_mobile_val[column] = predicted

In [176]:
df_fashion_val.to_csv('fashion_result_0303.csv')

In [100]:
### Open the json file

with open('../data/beauty_profile_train.json') as f:
    beauty_json = json.load(f)
with open('../data/fashion_profile_train.json') as f:
    fashion_json = json.load(f)
with open('../data/mobile_profile_train.json') as f:
    mobile_json = json.load(f)

In [101]:
beauty_column_map = {}
for column in beauty_feature_columns:
    beauty_column_map[column] = {v:k for k,v in beauty_json[column].items()}

fashion_column_map = {}
for column in fashion_feature_columns:
    fashion_column_map[column] = {v:k for k,v in fashion_json[column].items()}

mobile_column_map = {}
for column in mobile_feature_columns:
    mobile_column_map[column] = {v:k for k,v in mobile_json[column].items()}

In [103]:
df_beauty_val_text = df_beauty_val.copy()
for column in beauty_feature_columns:
    df_beauty_val_text.loc[:, column] = df_beauty_val_text[column].map(beauty_column_map[column])
    
df_fashion_val_text = df_fashion_val.copy()
for column in fashion_feature_columns:
    df_fashion_val_text.loc[:, column] = df_fashion_val_text[column].map(fashion_column_map[column])

df_mobile_val_text = df_mobile_val.copy()
for column in mobile_feature_columns:
    df_mobile_val_text.loc[:, column] = df_mobile_val_text[column].map(mobile_column_map[column])

In [114]:
df_fashion_val_text

,itemid,title,image_path,Collar Type,Sleeves,Pattern,Fashion Trend,Clothing Material
0,670968580,women casual o neck long sleeve lace patchwork...,fashion_image/b0ca9645c5c3527882d935f17b4c9579...,o neck,long sleeve,patchwork,sexy,lace
1,475599610,zaful long dress vintage wanita v neck lengan ...,fashion_image/c74af2636e9fe2686981decce3ca5b6c...,v neck,long sleeve,print,retro vintage,chiffon
2,1652671018,dress skater swing o neck tanpa lengan desain ...,fashion_image/1f2a2d91f5eb2d80f1dfdcf0b1f50169...,o neck,sleeveless,patchwork,retro vintage,lace
3,1705634904,ok dress slim sexy deep v neck lengan panjang ...,fashion_image/fdb6309635ebb5eac5df9c61984ffce5...,v neck,long sleeve,plain,sexy,lace
4,1789421087,noa korean lace embroidery womens crew neck lo...,fashion_image/578e5c74e97128cd25378ef0f0c021cf...,o neck,long sleeve,embroidery,korean,lace
5,1818944472,vintage women christmas lace patchwork o neck ...,fashion_image/dde2d4b3626a6924ea612d32315a5797...,o neck,sleeveless,patchwork,retro vintage,lace
6,1341150340,ready stock dress v neck tanpa lengan desain p...,fashion_image/d951fbeb548a5a5a2a307b56aa791bfa...,v neck,sleeveless,patchwork,party,lace
7,1021592916,women patchwork long sleeve lace o neck beach ...,fashion_image/4a5fe5c2019b4e466e843dbbe011ef4b...,o neck,long sleeve,patchwork,party,lace
8,1252778162,fashion wanita dress bodycon bandage pensil la...,fashion_image/aefb671d09d8ad9004c37103d0f9e4a3...,v neck,sleeveless,plain,sexy,lace
9,1309386587,fashion bohemian women lady dress chiffon chro...,fashion_image/52a94a1c04a1f8b6b30f737521d20fdc...,v neck,sleeveless,stripe,bohemian,chiffon


# Training DecisionTree (DT)

In [17]:
def build_DT_model():
    """
    Build the Decision Tree model
    """ 


    # Build pipeline
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', DecisionTreeClassifier(random_state=0))])


    # Define parameters for grid search
    parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                 'clf__max_depth':(1,2,3,4,5),
                 }


    # Grid search across our parameters, scoring by accuracy
    gs_clf = GridSearchCV(estimator=text_clf, param_grid=parameters, cv=5, iid=False, n_jobs=-1, scoring='accuracy')

    return gs_clf

In [22]:
def train_test_DT_model(category, column, X_train, y_train):
    DT_model = build_DT_model()
    DT_model.fit(X_train, y_train)
    # Evaluation
    best_params = DT_model.best_params_
    predicted = DT_model.predict(X_test)
    
    # Print the score
    best_params = best_params
    print('Best Parameters: ', best_params)
    train_acc = round(DT_model.best_score_, 2)
    print('Train Set Accuracy: ', train_acc)
    test_acc = round(np.mean(predicted == y_test), 2)
    print('Test Set Accuracy: ', test_acc)
    
    DT_train['result'][column] = {}
    DT_train['result'][column]['params'] = best_params
    DT_train['result'][column]['train_acc'] = train_acc
    DT_train['result'][column]['test_acc'] = test_acc
    
    with open ('DT_clf_07032019_DT_{}_{}.pkl'.format(category, column), 'wb') as f:
        pickle.dump(DT_model, f)
        
    return DT_model

In [21]:
##### Record the json training file for DT

DT_train = {}
DT_train['date'] = '07-03-2019'
DT_train['model'] = 'DecisionTree'
DT_train['model_details'] = {'split': 0.2, 'remarks':'None'}
DT_train['result'] = {}


In [23]:
fashion_feature_columns = ['Collar Type', 'Sleeves', 'Pattern', 'Fashion Trend', 'Clothing Material']


df_fashion_train = pd.read_csv('../data/fashion_data_info_train_competition.csv')

for column in fashion_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_fashion_train, column, 0.2)
    DT_model = train_test_DT_model('fashion', column, X_train, y_train)

Now processing for column: Collar Type
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.71
Test Set Accuracy:  0.71
Now processing for column: Sleeves
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.97
Test Set Accuracy:  0.97
Now processing for column: Pattern
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.76
Test Set Accuracy:  0.76
Now processing for column: Fashion Trend
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.79
Test Set Accuracy:  0.79
Now processing for column: Clothing Material
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.63
Test Set Accuracy:  0.64


In [27]:
DT_train

{'date': '07-03-2019',
 'model': 'DecisionTree',
 'model_details': {'split': 0.2, 'remarks': 'None'},
 'result': {'Collar Type': {'params': {'clf__max_depth': 5,
    'tfidf__use_idf': True,
    'vect__ngram_range': (1, 2)},
   'train_acc': 0.71,
   'test_acc': 0.71},
  'Sleeves': {'params': {'clf__max_depth': 5,
    'tfidf__use_idf': True,
    'vect__ngram_range': (1, 1)},
   'train_acc': 0.97,
   'test_acc': 0.97},
  'Pattern': {'params': {'clf__max_depth': 5,
    'tfidf__use_idf': True,
    'vect__ngram_range': (1, 1)},
   'train_acc': 0.76,
   'test_acc': 0.76},
  'Fashion Trend': {'params': {'clf__max_depth': 5,
    'tfidf__use_idf': True,
    'vect__ngram_range': (1, 1)},
   'train_acc': 0.79,
   'test_acc': 0.79},
  'Clothing Material': {'params': {'clf__max_depth': 5,
    'tfidf__use_idf': False,
    'vect__ngram_range': (1, 1)},
   'train_acc': 0.63,
   'test_acc': 0.64},
  'Brand': {'params': {'clf__max_depth': 5,
    'tfidf__use_idf': True,
    'vect__ngram_range': (1, 1)},
 

In [26]:
beauty_feature_columns = [ 'Brand', 'Colour_group', 'Benefits', 'Product_texture', 'Skin_type']

df_beauty_train = pd.read_csv('../data/beauty_data_info_train_competition.csv')


for column in beauty_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_beauty_train, column, 0.2)
    DT_model = train_test_DT_model('beauty', column, X_train, y_train)

Now processing for column: Brand


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.29
Test Set Accuracy:  0.29
Now processing for column: Colour_group
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.67
Test Set Accuracy:  0.68
Now processing for column: Benefits


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.55
Test Set Accuracy:  0.55
Now processing for column: Product_texture
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.88
Test Set Accuracy:  0.88
Now processing for column: Skin_type
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.48
Test Set Accuracy:  0.49


In [29]:
mobile_feature_columns = ['Operating System', 'Features',
       'Network Connections', 'Memory RAM', 'Brand', 'Warranty Period',
       'Storage Capacity', 'Color Family', 'Phone Model', 'Camera',
       'Phone Screen Size']

df_mobile_train = pd.read_csv('../data/mobile_data_info_train_competition.csv')



for column in mobile_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_mobile_train, column, 0.2)
    SGD_model = train_test_DT_model('mobile', column, X_train, y_train)

Now processing for column: Operating System


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.94
Test Set Accuracy:  0.94
Now processing for column: Features
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.49
Test Set Accuracy:  0.49
Now processing for column: Network Connections
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.83
Test Set Accuracy:  0.82
Now processing for column: Memory RAM
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.63
Test Set Accuracy:  0.63
Now processing for column: Brand


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.74
Test Set Accuracy:  0.74
Now processing for column: Warranty Period
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.81
Test Set Accuracy:  0.81
Now processing for column: Storage Capacity
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.86
Test Set Accuracy:  0.86
Now processing for column: Color Family


...

Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.67
Test Set Accuracy:  0.68
Now processing for column: Phone Model


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 1)}
Train Set Accuracy:  0.21
Test Set Accuracy:  0.21
Now processing for column: Camera


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 3 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': False, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.32
Test Set Accuracy:  0.32
Now processing for column: Phone Screen Size
Best Parameters:  {'clf__max_depth': 5, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 2)}
Train Set Accuracy:  0.4
Test Set Accuracy:  0.4


In [31]:
# Save the result
with open('DT_0703.json', 'w') as outfile:  
    json.dump(DT_train, outfile)

In [116]:
df_fashion_leak = pd.read_csv('../data/fashion_data_info_val_submission.csv')

In [118]:
df_fashion_leak.rename(columns={"Clothing Material": "Clothing Material True", "Pattern": "Pattern True",
                              "Sleeves":"Sleeves True", "Collar Type":"Collar Type True", "Fashion Trend": "Fashion Trend True" })

,Id,Clothing Material True,Pattern True,Sleeves True,Collar Type True,Fashion Trend True
0,0,lace,patchwork,long sleeve,o neck,sexy
1,1,chiffon,print,long sleeve,v neck,retro vintage
2,2,lace,patchwork,sleeveless,o neck,retro vintage
3,3,lace,plain,long sleeve,v neck,sexy
4,4,lace,embroidery,long sleeve,o neck,korean
5,5,lace,patchwork,sleeveless,o neck,retro vintage
6,6,lace,patchwork,sleeveless,v neck,party
7,7,lace,patchwork,long sleeve,o neck,party
8,8,lace,plain,sleeveless,v neck,sexy
9,9,chiffon,stripe,sleeveless,o neck,bohemian


In [141]:
df_fashion_val_text

,itemid,title,image_path,Collar Type,Sleeves,Pattern,Fashion Trend,Clothing Material
0,670968580,women casual o neck long sleeve lace patchwork...,fashion_image/b0ca9645c5c3527882d935f17b4c9579...,o neck,long sleeve,patchwork,sexy,lace
1,475599610,zaful long dress vintage wanita v neck lengan ...,fashion_image/c74af2636e9fe2686981decce3ca5b6c...,v neck,long sleeve,print,retro vintage,chiffon
2,1652671018,dress skater swing o neck tanpa lengan desain ...,fashion_image/1f2a2d91f5eb2d80f1dfdcf0b1f50169...,o neck,sleeveless,patchwork,retro vintage,lace
3,1705634904,ok dress slim sexy deep v neck lengan panjang ...,fashion_image/fdb6309635ebb5eac5df9c61984ffce5...,v neck,long sleeve,plain,sexy,lace
4,1789421087,noa korean lace embroidery womens crew neck lo...,fashion_image/578e5c74e97128cd25378ef0f0c021cf...,o neck,long sleeve,embroidery,korean,lace
5,1818944472,vintage women christmas lace patchwork o neck ...,fashion_image/dde2d4b3626a6924ea612d32315a5797...,o neck,sleeveless,patchwork,retro vintage,lace
6,1341150340,ready stock dress v neck tanpa lengan desain p...,fashion_image/d951fbeb548a5a5a2a307b56aa791bfa...,v neck,sleeveless,patchwork,party,lace
7,1021592916,women patchwork long sleeve lace o neck beach ...,fashion_image/4a5fe5c2019b4e466e843dbbe011ef4b...,o neck,long sleeve,patchwork,party,lace
8,1252778162,fashion wanita dress bodycon bandage pensil la...,fashion_image/aefb671d09d8ad9004c37103d0f9e4a3...,v neck,sleeveless,plain,sexy,lace
9,1309386587,fashion bohemian women lady dress chiffon chro...,fashion_image/52a94a1c04a1f8b6b30f737521d20fdc...,v neck,sleeveless,stripe,bohemian,chiffon


In [122]:
fashion = pd.merge(df_fashion_val_text, df_fashion_leak, left_index=True, right_index=True)

In [126]:
fashion_result = {}

for column in fashion_feature_columns:
    fashion_result[column] = fashion[['{}_x'.format(column), '{}_y'.format(column)]]
    fashion_result[column] = fashion_result[column].loc[fashion_result[column]['{}_x'.format(column)]!=fashion_result[column]['{}_y'.format(column)]]

In [139]:
fashion_result['Clothing Material']

,Clothing Material_x,Clothing Material_y
19,polyester,rayon
485,cotton,NaN
486,rayon,NaN
489,cotton,NaN
490,cotton,NaN
491,nylon,polyester
492,cotton,NaN
493,polyester,cotton
495,polyester,NaN
499,cotton,NaN


In [140]:
df_fashion_leak['Fashion Trend'].isna().sum()

32287

In [ ]:
#### Evaluation

# Evaluation
best_params = SGD_model.best_params_
predicted = SGD_model.predict(X_test)
print('Best Parameters: ', best_params)
print('Train Set Accuracy: ', round(SGD_model.best_score_, 2))
print('Test Set Accuracy: ', round(np.mean(predicted == y_test), 2))
print('Train set Accuracy (cross val): ', cross_val_score(SGD_model, X_train, y_train, cv=5, scoring='accuracy'))
print(classification_report(y_test, predicted))
print(confusion_matrix(y_test, predicted))

In [147]:
df_mobile_val_text

,itemid,title,image_path,Operating System,Features,Network Connections,Memory RAM,Brand,Warranty Period,Storage Capacity,Color Family,Phone Model,Camera,Phone Screen Size
0,1520485457,new promo iphone 5s 16gb gold ex resmi ibox,mobile_image/876d4a1fe29e056855fa6f9643757b1c.jpg,ios,fingerprint sensor,4g,16gb,apple,3 months,32gb,gold,apple iphone 5s,dua slot,3.6 to 4 inches
1,1520516704,new promo xiaomi note 5a prime 3 32gb tam,mobile_image/e2b902b7cd35cd50f061d8e2a3ba7178.jpg,android,touchscreen,2g,3gb,xiaomi,1 year,32gb,gold,xiaomi note 5a prime,16mp,5.1 to 5.5 inches
2,1520726573,big produk super murah oppo f5 4gb 32gb fullset,mobile_image/785c03cb1a21a1efedb8c1bbbdb81035.jpg,android,fingerprint sensor,4g,4gb,oppo,1 year,32gb,gold,oppo f5,16mp,less than 3.5 inches
3,1523303826,pocophone xiaomi f1 ram 6gb rom 128gb,mobile_image/08bb7a43ae0fafd568ad54db908bac0b.jpg,android,touchscreen,4g,6gb,xiaomi,1 year,128gb,black,xiaomi pocophone f1,dua slot,4.6 to 5 inches
4,1524467616,advan g2,mobile_image/970134be2e5961bda177e9761e1cf6ab.jpg,android,fingerprint sensor,4g,3gb,advan,1 month,32gb,white,oppo f9,13mp,more than 5.6 inches
5,1524876429,samsung galaxy a6,mobile_image/ce06b7eb04ff32e0ba9bd8de93b47955.jpg,android,touchscreen,4g,3gb,samsung,18 months,32gb,black,samsung galaxy a6,16mp,more than 5.6 inches
6,1525744340,promo akhir bulan oppo f7 diamond black limite...,mobile_image/9c4ed6c4eb9fe1bbfdb873f527896506.jpg,android,gps,4g,4gb,oppo,2 month,64gb,black,oppo f7,16mp,4.6 to 5 inches
7,1529179598,vivo v9 pro ram 6gb rom 64gb 6,mobile_image/8af91359e6b7f04d8099748d8fbdd200.jpg,android,waterproof,4g,6gb,vivo,1 month,64gb,red,vivo v9,13mp,more than 5.6 inches
8,1529634640,unik telephone panasonic wireless kx tg1611 bl...,mobile_image/f50c21529d6d99a7aa9bafdb56fbe3a8.jpg,android,fingerprint sensor,3g,4gb,hp,1 year,64gb,black,apple iphone 7,single camera,4.1 to 4.5 inches
9,1529634708,promo telepon wireless panasonic kx tg1611 all...,mobile_image/a3f4f39dad3ec46faf58d9f0e8d4d0f5.jpg,android,fingerprint sensor,3g,4gb,hp,1 year,32gb,white,samsung galaxy note 8,single camera,4.1 to 4.5 inches


In [145]:
beauty_json['Brand']

{'selection': 0,
 'bio hair': 124,
 'katy perry': 2,
 'cw 666': 3,
 'random': 4,
 'wet n wild': 5,
 'chanel': 212,
 'pomparians': 6,
 'xl professionnel': 36,
 'herborist': 9,
 'cantiqa kemiri': 10,
 'catok haidi': 11,
 'dettol': 12,
 'hadalabo': 13,
 'pepsodent': 14,
 'pratista': 103,
 'dolce gabbana': 368,
 'evangeline': 16,
 'cocco': 17,
 'scarlett': 18,
 'colourpop': 19,
 'little baby': 20,
 'avoskin': 21,
 'moodmatcher': 22,
 'sasha': 23,
 'davidoff': 24,
 'makarizo viola': 25,
 'miniso': 108,
 'kemei': 28,
 'bebold': 140,
 'erto s': 30,
 'rudy hadisuwarno': 31,
 'kirkland signature': 32,
 'haidi': 33,
 'marc jacobs': 34,
 'jo malone': 35,
 'ellips': 7,
 'l oreal paris': 104,
 'real techniques': 38,
 'musk by lilian ashley': 39,
 'sonar': 40,
 'olay': 41,
 'ql': 42,
 'zwitsbaby': 271,
 'natur': 45,
 'satto': 46,
 'revlon': 47,
 'emina': 48,
 'envygreen': 49,
 'peripera': 50,
 'natrol': 51,
 'champagne': 63,
 'glamglow': 52,
 'elise': 53,
 'paris hilton': 376,
 'la rive': 54,
 'chri

In [ ]:
df_beauty_train = pd.read_csv('./data/beauty_data_info_train_competition.csv')


# Train Random Forest

In [149]:
def train_RF_model():
    """
    Random forest model
    """
    X_train, X_test, y_train, y_test = train_test_arrays # Unpack data


    # Preprocess each sentence in the train and test sets
    X_train, X_test = np.array([preprocessor(x) for x in X_train]), np.array([preprocessor(x) for x in X_test])

    # Build pipeline
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', RandomForestClassifier(n_estimators=750))])


    # Define parameters for grid search
    parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
                'tfidf__use_idf': (True, False),}


    # Grid search across our parameters, scoring by accuracy
    gs_clf = GridSearchCV(estimator=text_clf, param_grid=parameters, cv=5, iid=False, n_jobs=-1, scoring='accuracy')
    gs_clf.fit(X_train, y_train)

    return gs_clf, X_test

In [237]:
df_sample

,id,tagging
0,370855998_Benefits,3 2
1,370855998_Brand,246 98
2,370855998_Colour_group,29 26
3,370855998_Product_texture,3 4
4,370855998_Skin_type,7 7
5,637234604_Benefits,4 2
6,637234604_Brand,338 248
7,637234604_Colour_group,42 15
8,637234604_Product_texture,7 4
9,637234604_Skin_type,5 0


# Train XGBoost

In [178]:
df_fashion_submit = pd.DataFrame(fashion_submit_dict)

In [10]:
for column in beauty_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_beauty_train, column, 0.2)
    X_train = X_train.as_matrix()
    y_train = y_train.as_matrix()
    print (X_train.shape)
    model = XGBClassifier()
    model.fit(X_train[:,np.newaxis],y_train)
    

Now processing for column: Brand
(190502,)


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  """
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:6: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  


ValueError: could not convert string to float: 'nars light reflecting translucent powder'

In [21]:
def build_XGB_model():
    """
    Random XGBoost model
    """
    
    # Build pipeline
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('xgb', XGBClassifier(n_estimators=2, silent = False, n_jobs = -1,
                                               objective = 'softmax'))])


    # Define parameters for grid search
    parameters = {'vect__ngram_range': [(1, 1), (1, 2), (1, 3)],
                'tfidf__use_idf': (True, False),
                 'xgb__learning_rate': [0.01, 0.1]}
    


    # Grid search across our parameters, scoring by accuracy
    gs_clf = GridSearchCV(estimator=text_clf, param_grid=parameters, cv=5, iid=False, n_jobs=-1, scoring='accuracy')
    gs_clf.fit(X_train, y_train)

    return gs_clf

In [24]:
# beauty_feature_columns = [ 'Brand', 'Colour_group', 'Benefits', 'Product_texture', 'Skin_type']

beauty_feature_columns = ['Brand']
df_beauty_train = pd.read_csv('../data/beauty_data_info_train_competition.csv')

# df_beauty_train = df_beauty_train.head(n=1500)
for column in beauty_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data(df_beauty_train, column, 0.2)
    XGB_model = train_test_XGB_model('beauty', column, X_train, y_train)

Now processing for column: Brand


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 1 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


KeyboardInterrupt: 

In [12]:
def train_test_XGB_model(category, column, X_train, y_train):
    XGB_model = build_XGB_model()
    XGB_model.fit(X_train, y_train)
    # Evaluation
    best_params = XGB_model.best_params_
    predicted = XGB_model.predict(X_test)
    
    # Print the score
    best_params = best_params
    print('Best Parameters: ', best_params)
    train_acc = round(XGB_model.best_score_, 2)
    print('Train Set Accuracy: ', train_acc)
    test_acc = round(np.mean(predicted == y_test), 2)
    print('Test Set Accuracy: ', test_acc)
    
    with open ('XGB_clf_09032019_DT_{}_{}.pkl'.format(category, column), 'wb') as f:
        pickle.dump(XGB_model, f)
        
    return XGB_model

In [162]:
df_beauty_submit = pd.DataFrame(beauty_submit_dict)

In [210]:
df_fashion_submit['tagging'].dtypes

dtype('float64')

In [224]:
final_submit = df_fashion_submit.append([df_mobile_submit, df_beauty_submit])

In [240]:
final_submit.iloc[1]['tagging']

'1.0 1'

In [250]:
for index, row in final_submit.iterrows():
    row['tagging'] = row['tagging']+" 1"

In [251]:
final_submit['tagging']

0          3 1
1          1 1
2          1 1
3          0 1
4          8 1
5          0 1
6          8 1
7          8 1
8          1 1
9          3 1
10         0 1
11         1 1
12        13 1
13         8 1
14         8 1
15         3 1
16         8 1
17        13 1
18         1 1
19         1 1
20         8 1
21         3 1
22         1 1
23         1 1
24         8 1
25         8 1
26         1 1
27         3 1
28         8 1
29         8 1
          ... 
382695     0 1
382696     2 1
382697     0 1
382698     0 1
382699     0 1
382700     0 1
382701     0 1
382702     0 1
382703     0 1
382704     0 1
382705     0 1
382706     0 1
382707     0 1
382708     0 1
382709     2 1
382710     0 1
382711     3 1
382712     6 1
382713     0 1
382714     2 1
382715     0 1
382716     0 1
382717     4 1
382718     0 1
382719     0 1
382720     0 1
382721     0 1
382722     0 1
382723     0 1
382724     0 1
Name: tagging, Length: 1174802, dtype: object

# Train Voting Classifier

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.ensemble import RandomForestClassifier, VotingClassifier

In [8]:
names = ["Nearest Neighbors", "Linear SVM", "RBF SVM", "Gaussian Process",
         "Decision Tree", "Random Forest", "Neural Net", "AdaBoost",
         "Naive Bayes", "QDA"]

In [50]:
### Train the voting model
def train_test_voting_model(category, target_column, X_train, y_train):
    voting_model = build_voting_model()
    voting_model.fit(X_train, y_train)
    # Evaluation
    best_params = voting_model.best_params_
    predicted = voting_model.predict(X_test)
    print('Voting Classifiers: ', voting_model.get_params())
    print('Best Parameters: ', best_params)
    print('Train Set Accuracy: ', round(voting_model.best_score_, 2))
    print('Test Set Accuracy: ', round(np.mean(predicted == y_test), 2))
    
    with open ('voting_clf_17032019_{}_{}.pkl'.format(category, target_column), 'wb') as f:
        pickle.dump(voting_model, f)
        
    return voting_model


In [52]:
def build_voting_model():
    """
    Build the Voting model
    """ 

    LOG_clf = LogisticRegression(solver = 'lbfgs', multi_class = 'multinomial', random_state = 1)
    RF_clf = RandomForestClassifier(n_estimators = 20, random_state = 1)
    NB_clf = GaussianNB()


    vot_clf = VotingClassifier(estimators = [
        ('LOG_clf', LOG_clf),
        ('RF_clf', RF_clf),
        ('NB_clf', NB_clf)
    ])


    text_clf = Pipeline([
        ('vect', CountVectorizer()),
        ('tfidf', TfidfTransformer()),
        ('to_dense', DenseTransformer()),
        ('vot_clf', vot_clf)])
    # Define parameters for grid search
    parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False)
                 }


    # Grid search across our parameters, scoring by accuracy
    gs_clf = GridSearchCV(estimator=text_clf, param_grid=parameters, cv=5, iid=False, n_jobs=2, scoring='accuracy')

    return gs_clf

In [32]:
def build_SGD_model():
    """
    Build the SGD model
    """ 

    # Build pipeline
    text_clf = Pipeline([('vect', CountVectorizer()),
                         ('tfidf', TfidfTransformer()),
                         ('clf', SGDClassifier(loss='modified_huber', max_iter=1000, tol=1e-3, class_weight='balanced'))])


    # Define parameters for grid search
    parameters = {'vect__ngram_range': [(1, 1), (1, 2)],
                'tfidf__use_idf': (True, False),
                'clf__alpha': (1e-1, 1e-5)}


    # Grid search across our parameters, scoring by accuracy
    gs_clf = GridSearchCV(estimator=text_clf, param_grid=parameters, cv=5, iid=False, n_jobs=-1, scoring='accuracy')

    return gs_clf

In [43]:
from sklearn.base import TransformerMixin
class DenseTransformer(TransformerMixin):

    def fit(self, X, y=None, **fit_params):
        return self

    def transform(self, X, y=None, **fit_params):
        return X.todense()

In [53]:
mobile_feature_columns = ['Operating System', 'Features',
       'Network Connections', 'Memory RAM', 'Brand', 'Warranty Period',
       'Storage Capacity', 'Color Family', 'Phone Model', 'Camera',
       'Phone Screen Size']

df_mobile_train = pd.read_csv('../data/mobile_data_info_train_competition.csv')

#df_mobile_train = df_mobile_train.head(n=1000)
for column in mobile_feature_columns:
    print ("Now processing for column:", column)
    # Unpack data
    X_train,X_test,y_train,y_test = train_test_data_title(df_mobile_train, column, 0.2)
    voting_model = train_test_voting_model('mobile', column, X_train, y_train)
    

Now processing for column: Operating System


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 4 members, which is too few. The minimum number of members in any class cannot be less than n_splits=5.
  % (min_groups, self.n_splits)), Warning)


TerminatedWorkerError: A worker process managed by the executor was unexpectedly terminated. This could be caused by a segmentation fault while calling the function or by an excessive memory usage causing the Operating System to kill the worker. The exit codes of the workers are {SIGKILL(-9)}

# TO-DO

In [ ]:
# Check invalid RAM

In [110]:
# Read csv
df_m = pd.read_csv('../data_complementary/webcrawler/utils/testoutput.txt')

In [111]:
df_m

,itemid,title,image_path,Operating System,Features,Network Connections,Memory RAM,Brand,Warranty Period,Storage Capacity,Color Family,Phone Model,Camera,Phone Screen Size
0,2346660,apple iphone 4s back glass spare part original...,mobile_image/a9c8f0fdd6587deed197634066cf7eee.jpg,1.0,NaN,NaN,1.0,2.0,NaN,NaN,12.0,1526.0,NaN,2.0
1,2816338,iphone 4s 64gb white,mobile_image/3b9a11608551b11b9330268e0d055e01.jpg,NaN,NaN,NaN,NaN,2.0,NaN,3.0,12.0,NaN,NaN,NaN
2,2847602,samsung sm b310e piton dual sim,mobile_image/1d719e936841a83c165da620f927de68.jpg,NaN,NaN,NaN,NaN,43.0,13.0,NaN,12.0,NaN,NaN,NaN
3,3116949,samsung caramel gt e1272 dual sim 32 mb putih,mobile_image/1d35a74d90df6cf4a02e6a5df9e9ff29.jpg,NaN,NaN,NaN,NaN,43.0,13.0,NaN,12.0,NaN,NaN,NaN
4,3794648,garskin sony experia z z1 z2 ultra,mobile_image/5556577b09539a9c0db0d00e0f171e2d.jpg,6.0,NaN,0.0,NaN,38.0,NaN,NaN,NaN,NaN,NaN,NaN
5,4980072,lcd xiaomi redmi 4+touchscreen,mobile_image/504bbab21ede157e6e3f1b93e6b6484c.jpg,NaN,0.0,NaN,NaN,33.0,NaN,NaN,NaN,376.0,NaN,NaN
6,5769008,samsung caramel gt e1272 dual sim 32mb black,mobile_image/e088ca5ebb1ab5ba90a8cff8f9c4f791.jpg,NaN,NaN,NaN,NaN,43.0,13.0,NaN,10.0,NaN,NaN,NaN
7,9503620,iphone 4g 8gb,mobile_image/a23f0381039e5595559be27db3271d2f.jpg,NaN,NaN,0.0,3.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN
8,17937158,blackberry torch 1 9800 gsm garansi distributo...,mobile_image/7803e0e63b5972e14b6ff564679f941c.jpg,NaN,0.0,NaN,NaN,51.0,2.0,NaN,12.0,NaN,NaN,NaN
9,21715801,samsung keystone 3 sm b109e,mobile_image/a5360d928a586de4b7dc5a8463f9fc26.jpg,2.0,NaN,1.0,7.0,43.0,NaN,NaN,NaN,559.0,NaN,2.0
